#### Import Data CSV

In [ ]:
import pandas as pd
df = pd.read_csv('dirty_cafe_sales.csv')
print (df.shape, df.columns)
# print(df.head(3))
# print(df.tail(3))
# print(df.describe())

# print (df.columns)


(10000, 8) Index(['Transaction ID', 'Item', 'Quantity', ' Price Per Unit', ' Total Spent',
       'Payment Method', 'Location', 'Transaction Date'],
      dtype='object')


#### Mengganti nama kolom agar konsisten snake_case

In [ ]:
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(' ', '_')
    )  # Remove leading/trailing whitespace from column names
print(df.columns.tolist())# Display the cleaned column names

['transaction_id', 'item', 'quantity', 'price_per_unit', 'total_spent', 'payment_method', 'location', 'transaction_date']


#### Mencari data duplikat

In [ ]:
print('Data duplicate =', df.duplicated().sum(), '\n')  # Display the number of duplicate rows
df = df.drop_duplicates()  # Remove duplicate rows if any

Data duplicate = 0 



#### Memperbaiki baris data yang missing value 

In [ ]:
print('kolom yang memiliki value null', '\n')  # Check if any column has null values
print(df.isna().sum(), '\n')  # Display the count of missing values in each column

# df['quantity'] = df['quantity'].fillna(0)  # Fill missing values in 'quantity' column with 0
# df['payment_method'] = df['payment_method'].fillna('UNKNOWN')  # Fill missing values in 'payment_method' column with 'unknown'
# df['location'] = df['location'].fillna('UNKNOWN')  # Fill missing values in 'location' column with 'unknown'
# df['price_per_unit'] = df['price_per_unit'].fillna(df['price_per_unit'].mean())  # Fill missing values in 'price_per_unit' column with the mean of that column

print('Data sebelum menghapus baris yang memiliki missing value =', df.shape, '\n')  # Display the shape of the DataFrame before removing rows with missing values
df = df.dropna()  # Remove rows with any missing values
print('Data setelah menghapus baris yang memiliki missing value =', df.shape, '\n')  # Display the shape of the DataFrame after removing rows with missing values
print(df.isna().sum(), '\n')  # Display the count of missing values in each column
print(df.head(15))  # Display the first 15 rows of the DataFrame

kolom yang memiliki value null 

transaction_id         0
item                 333
quantity             138
price_per_unit       179
total_spent          173
payment_method      2579
location            3265
transaction_date     159
dtype: int64 

Data sebelum menghapus baris yang memiliki missing value = (10000, 8) 

Data setelah menghapus baris yang memiliki missing value = (4550, 8) 

transaction_id      0
item                0
quantity            0
price_per_unit      0
total_spent         0
payment_method      0
location            0
transaction_date    0
dtype: int64 

   transaction_id      item quantity price_per_unit total_spent  \
0     TXN_1961373    Coffee        2            2.0         4.0   
1     TXN_4977031      Cake        4            3.0        12.0   
2     TXN_4271903    Cookie        4            1.0       ERROR   
3     TXN_7034554     Salad        2            5.0        10.0   
4     TXN_3160411    Coffee        2            2.0         4.0   
6     TXN_443321

#### Memperbaiki data yang tidak sesuai

In [ ]:
df[['quantity', 'total_spent', 'price_per_unit']] = df[['quantity', 'total_spent', 'price_per_unit']].replace(['ERROR','UNKNOWN'], pd.NA)  # Replace 'ERROR' with '0' in specified columns



df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')  # Convert 'quantity' to numeric, coercing errors to NaN
df['total_spent'] = pd.to_numeric(df['total_spent'], errors='coerce')  # Convert 'total_spent' to numeric, coercing errors to NaN
df['price_per_unit'] = pd.to_numeric(df['price_per_unit'], errors='coerce')  # Convert 'price_per_unit' to numeric, coercing errors to NaN

# Isi nilai quantity yang kosong jika total_spent dan price_per_unit tersedia
mask_q = df['quantity'].isna() & df['total_spent'].notna() & df['price_per_unit'].notna()
df.loc[mask_q, 'quantity'] = df.loc[mask_q, 'total_spent'] / df.loc[mask_q, 'price_per_unit']

# Isi nilai price_per_unit yang kosong jika total_spent dan quantity tersedia
mask_p = df['price_per_unit'].isna() & df['total_spent'].notna() & df['quantity'].notna()
df.loc[mask_p, 'price_per_unit'] = df.loc[mask_p, 'total_spent'] / df.loc[mask_p, 'quantity']

# Isi nilai total_spent yang kosong jika quantity dan price_per_unit tersedia
mask_t = df['total_spent'].isna() & df['quantity'].notna() & df['price_per_unit'].notna()
df.loc[mask_t, 'total_spent'] = df.loc[mask_t, 'quantity'] * df.loc[mask_t, 'price_per_unit']

print(df.head(25))  # Display the first 15 rows of the DataFrame after cleaning

   transaction_id      item  quantity  price_per_unit  total_spent  \
0     TXN_1961373    Coffee       2.0             2.0          4.0   
1     TXN_4977031      Cake       4.0             3.0         12.0   
2     TXN_4271903    Cookie       4.0             1.0          4.0   
3     TXN_7034554     Salad       2.0             5.0         10.0   
4     TXN_3160411    Coffee       2.0             2.0          4.0   
6     TXN_4433211   UNKNOWN       3.0             3.0          9.0   
7     TXN_6699534  Sandwich       4.0             4.0         16.0   
10    TXN_2548360     Salad       5.0             5.0         25.0   
11    TXN_3051279  Sandwich       2.0             4.0          8.0   
12    TXN_7619095  Sandwich       2.0             4.0          8.0   
15    TXN_2847255     Salad       3.0             5.0         15.0   
17    TXN_6769710     Juice       2.0             3.0          6.0   
18    TXN_8876618      Cake       5.0             3.0         15.0   
19    TXN_3709394   

#### Hapus baris dengan nilai 'Error' dan 'Unknown'

In [ ]:
# df.drop[df['item'] ['ERROR', 'UNKNOWN'])].reset_index(drop=True)
df.drop(df[df['item'].isin(['UNKNOWN', 'ERROR']) ].index, inplace=True)
df.drop(df[df['location'].isin(['UNKNOWN', 'ERROR']) ].index, inplace=True)
df.drop(df[df['payment_method'].isin(['UNKNOWN', 'ERROR']) ].index, inplace=True)
df.drop(df[df['transaction_date'].isin(['UNKNOWN', 'ERROR']) ].index, inplace=True)

print(df.head(15))  # Display the first 15 rows of the DataFrame after cleaning

   transaction_id      item  quantity  price_per_unit  total_spent  \
0     TXN_1961373    Coffee       2.0             2.0          4.0   
1     TXN_4977031      Cake       4.0             3.0         12.0   
2     TXN_4271903    Cookie       4.0             1.0          4.0   
4     TXN_3160411    Coffee       2.0             2.0          4.0   
10    TXN_2548360     Salad       5.0             5.0         25.0   
12    TXN_7619095  Sandwich       2.0             4.0          8.0   
15    TXN_2847255     Salad       3.0             5.0         15.0   
17    TXN_6769710     Juice       2.0             3.0          6.0   
19    TXN_3709394     Juice       4.0             3.0         12.0   
20    TXN_3522028  Smoothie       5.0             4.0         20.0   
21    TXN_3567645  Smoothie       4.0             4.0         16.0   
22    TXN_5132361  Sandwich       3.0             4.0         12.0   
24    TXN_9400181  Sandwich       5.0             4.0         20.0   
26    TXN_5183041   

In [ ]:
# Load data csv
df = pd.read_csv('cleaned_data_fix.csv')

# Definisikan harga per item sesuai kebutuhan
price_dict = {
    'Coffee': 2,
    'Cake': 3,
    'Cookie': 1,
    'Salad': 5,
    'Sandwich': 4,
    'Juice': 3,
    'Smoothie': 4,
    'Tea': 1.5
}

# Cek nilai kosong pada price_per_unit sebelum diisi
print("Data dengan price_per_unit kosong sebelum diisi:")
print(df[df['price_per_unit'].isnull()])

# Hapus baris yang quantity dan total_spent kosong (NaN)
df = df.dropna(subset=['quantity', 'total_spent'], how='all')

# Isi nilai price_per_unit yang kosong berdasarkan nama item
df['price_per_unit'] = df.apply(
    lambda row: price_dict[row['item']] if pd.isna(row['price_per_unit']) else row['price_per_unit'],
    axis=1
)

# Hitung total_spent = quantity * price_per_unit untuk semua baris
df['total_spent'] = df['quantity'] * df['price_per_unit']

# Cek hasil pengisian
print("\nData setelah price_per_unit diisi:")
print(df[df['price_per_unit'].isnull() == False].head(10))


Data dengan price_per_unit kosong sebelum diisi:
     transaction_id      item  quantity  price_per_unit  total_spent  \
1216    TXN_2857444  Smoothie       1.0             NaN          NaN   
1368    TXN_6424202    Cookie       2.0             NaN          NaN   
2499    TXN_5118799    Cookie       2.0             NaN          NaN   
2508    TXN_2253622  Sandwich       5.0             NaN          NaN   
2796    TXN_6105807  Smoothie       3.0             NaN          NaN   
2928    TXN_1525583  Sandwich       3.0             NaN          NaN   
2992    TXN_7764304      Cake       NaN             NaN          3.0   

      payment_method  location transaction_date  
1216            Cash  Takeaway       2023-05-10  
1368     Credit Card  In-store       2023-11-20  
2499            Cash  Takeaway       2023-04-23  
2508  Digital Wallet  Takeaway       2023-09-30  
2796     Credit Card  Takeaway       2023-01-18  
2928            Cash  Takeaway       2023-05-20  
2992     Credit Card  Ta

In [ ]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])  # Convert 'transaction_date' to datetime format, coercing errors to NaT

#### Simpan DataFrame hasil cleaning ke file CSV

In [ ]:
df.to_csv('cleaned_data_price_filled.csv', index=False)
print("\nFile dengan kolom price_per_unit yang diisi sudah disimpan: 'cleaned_data_price_filled.csv'")


File dengan kolom price_per_unit yang diisi sudah disimpan: 'cleaned_data_price_filled.csv'
